In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# 加载数据

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
print(f"训练集长度:{len(train_df)}, 测试集长度:{len(test_df)}")
# pd.set_option('display.max_columns', None)
# print(train_df.head())

训练集长度:1460, 测试集长度:1459


# 特征工程

In [3]:
## 查看缺失值：
# pd.set_option('display.max_rows', None)
# print(train_df.isnull().sum())
# print(test_df.isnull().sum())

## 缺失值处理

### 区域与设施

In [4]:
### 区域分类：MSZoning，测试集缺失4->众数插补
MSZoning_mode = train_df["MSZoning"].mode()[0]
test_df.loc[:,"MSZoning"] = test_df["MSZoning"].fillna(MSZoning_mode)

### 房子同街道之间的距离： LotFrontage 训练集缺失259，测试集缺失227-》中位数插补
LotFrontage_median_imputer = SimpleImputer(strategy='median')
train_df.loc[:, "LotFrontage"] = LotFrontage_median_imputer.fit_transform(train_df[["LotFrontage"]])
test_df.loc[:,"LotFrontage"] = LotFrontage_median_imputer.transform(test_df[["LotFrontage"]])

### 小道的路面类型 Alley 训练集缺失1369 测试集缺失1352, 缺失量过多, 用None表示
train_df.loc[:,"Alley"] = train_df["Alley"].fillna("None")
test_df.loc[:,"Alley"] = test_df["Alley"].fillna("None")

### 配套公用设施类型 Utilities 测试集缺失2->众数插补
Utilities_mode = train_df["Utilities"].mode()[0]
test_df.loc[:,"Utilities"] = test_df["Utilities"].fillna(Utilities_mode)

### 建材

In [5]:
### 外立面材料 Exterior1st 测试集缺失1->众数插补
Exterior1st_mode = train_df["Exterior1st"].mode()[0]
test_df.loc[:,"Exterior1st"] = test_df["Exterior1st"].fillna(Exterior1st_mode)

### 外立面材料2 Exterior2nd 测试集缺失1->众数插补
Exterior2nd_mode = train_df["Exterior2nd"].mode()[0]
test_df.loc[:,"Exterior2nd"] = test_df["Exterior2nd"].fillna(Exterior2nd_mode)

### 装饰石材类型 MasVnrType 训练集缺失872, 测试集缺失894-》众数插补？缺失的也很多
MasVnrType_mode = train_df["MasVnrType"].mode()[0]
train_df.loc[:,"MasVnrType"] = train_df["MasVnrType"].fillna(MasVnrType_mode)
test_df.loc[:,"MasVnrType"] = test_df["MasVnrType"].fillna(MasVnrType_mode)

### 装饰石材面积 MasVnrArea 训练集缺失8, 测试集缺失15-》中位数插补
MasVnrArea_median_imputer = SimpleImputer(strategy='median')
train_df.loc[:, "MasVnrArea"] = MasVnrArea_median_imputer.fit_transform(train_df[["MasVnrArea"]])
test_df.loc[:,"MasVnrArea"] = MasVnrArea_median_imputer.transform(test_df[["MasVnrArea"]])

### 地下室

In [6]:
# 评估地下室层高情况 BsmtQual 训练集缺失37 测试集缺失44-》众数插补
BsmtQual_mode = train_df["BsmtQual"].mode()[0]
train_df.loc[:, "BsmtQual"] = train_df["BsmtQual"].fillna(BsmtQual_mode)
test_df.loc[:,"BsmtQual"] = test_df["BsmtQual"].fillna(BsmtQual_mode)

# 地下室总体情况 BsmtCond 训练集缺失37 测试集缺失45-》众数插补
BsmtCond_mode = train_df["BsmtCond"].mode()[0]
train_df.loc[:, "BsmtCond"] = train_df["BsmtCond"].fillna(BsmtCond_mode)
test_df.loc[:,"BsmtCond"] = test_df["BsmtCond"].fillna(BsmtCond_mode)

# 地下室出口或者花园层的墙面 BsmtExposure       训练集缺失38 测试集缺失44-》众数插补
BsmtExposure_mode = train_df["BsmtExposure"].mode()[0]
train_df.loc[:, "BsmtExposure"] = train_df["BsmtExposure"].fillna(BsmtExposure_mode)
test_df.loc[:,"BsmtExposure"] = test_df["BsmtExposure"].fillna(BsmtExposure_mode)

# 地下室区域质量 BsmtFinType1       训练集缺失37 测试集缺失42-》众数插补
BsmtFinType1_mode = train_df["BsmtFinType1"].mode()[0]
train_df.loc[:, "BsmtFinType1"] = train_df["BsmtFinType1"].fillna(BsmtFinType1_mode)
test_df.loc[:,"BsmtFinType1"] = test_df["BsmtFinType1"].fillna(BsmtFinType1_mode)
# 二次完工面积质量（如果有） BsmtFinType2       训练集缺失38，测试集缺失42-》众数插补
BsmtFinType2_mode = train_df["BsmtFinType2"].mode()[0]
train_df.loc[:, "BsmtFinType2"] = train_df["BsmtFinType2"].fillna(BsmtFinType2_mode)
test_df.loc[:,"BsmtFinType2"] = test_df["BsmtFinType2"].fillna(BsmtFinType2_mode)
# 1完工面积 BsmtFinSF1          测试集缺失1-》中位数插补
BsmtFinSF1_median_imputer = SimpleImputer(strategy='median')
train_df.loc[:, "BsmtFinSF1"] = BsmtFinSF1_median_imputer.fit_transform(train_df[["BsmtFinSF1"]])
test_df.loc[:,"BsmtFinSF1"] = BsmtFinSF1_median_imputer.transform(test_df[["BsmtFinSF1"]])
# 2完工面积 BsmtFinSF2          测试集缺失1-》中位数插补
BsmtFinSF2_median_imputer = SimpleImputer(strategy='median')
train_df.loc[:, "BsmtFinSF2"] = BsmtFinSF2_median_imputer.fit_transform(train_df[["BsmtFinSF2"]])
test_df.loc[:,"BsmtFinSF2"] = BsmtFinSF2_median_imputer.transform(test_df[["BsmtFinSF2"]])

# 地下室区域未完工面积 BsmtUnfSF           测试集缺失1-》中位数插补
BsmtUnfSF_median_imputer = SimpleImputer(strategy='median')
train_df.loc[:, "BsmtUnfSF"] = BsmtUnfSF_median_imputer.fit_transform(train_df[["BsmtUnfSF"]])
test_df.loc[:,"BsmtUnfSF"] = BsmtUnfSF_median_imputer.transform(test_df[["BsmtUnfSF"]])

# 地下室总体面积 TotalBsmtSF         测试集缺失1-》中位数插补
TotalBsmtSF_median_imputer = SimpleImputer(strategy='median')
train_df.loc[:, "TotalBsmtSF"] = BsmtUnfSF_median_imputer.fit_transform(train_df[["TotalBsmtSF"]])
test_df.loc[:,"TotalBsmtSF"] = BsmtUnfSF_median_imputer.transform(test_df[["TotalBsmtSF"]])

# 地下室全浴室数量 BsmtFullBath  测试集缺失2-》众数插补(取值只有0，1，2，3)
BsmtFullBath_mode = train_df["BsmtFullBath"].mode()[0]
test_df.loc[:,"BsmtFullBath"] = test_df["BsmtFullBath"].fillna(BsmtFullBath_mode)

# 地下室半浴室数量 BsmtHalfBath 测试集缺失2-》众数插补(取值只有0，1，2)
BsmtHalfBath_mode = train_df["BsmtHalfBath"].mode()[0]
test_df.loc[:,"BsmtHalfBath"] = test_df["BsmtHalfBath"].fillna(BsmtHalfBath_mode)

### 厨电配套

In [7]:
### 厨房质量 KitchenQual         测试集缺失1-》众数插补
KitchenQual_mode = train_df["KitchenQual"].mode()[0]
test_df.loc[:,"KitchenQual"] = test_df["KitchenQual"].fillna(KitchenQual_mode)

### 功能性评级 Functional          测试集缺失2-》众数插补
Functional_mode = train_df["Functional"].mode()[0]
test_df.loc[:,"Functional"] = test_df["Functional"].fillna(Functional_mode)

### 电力系统 Electrical          训练集缺失 1-》众数插补
Electrical_mode = train_df["Electrical"].mode()[0]
train_df.loc[:,"Electrical"] = train_df["Electrical"].fillna(Electrical_mode)

### 壁炉质量 FireplaceQu       训练集缺失690，测试集缺失730
train_df.loc[:,"FireplaceQu"] = train_df["FireplaceQu"].fillna("None")
test_df.loc[:,"FireplaceQu"] = test_df["FireplaceQu"].fillna("None")

### 车库

In [8]:
### 车库位置 GarageType 训练集缺失81，测试集缺失76-》众数插补
GarageType_mode = train_df["GarageType"].mode()[0]
train_df.loc[:,"GarageType"] = train_df["GarageType"].fillna(GarageType_mode)
test_df.loc[:,"GarageType"] = test_df["GarageType"].fillna(GarageType_mode)

### 车库建造时间 GarageYrBlt  训练集缺失81，测试集缺失78 -> 先用unknown表示，后续需要构造特征
train_df.loc[:,"GarageYrBlt"] = train_df["GarageYrBlt"].fillna("None")
test_df.loc[:,"GarageYrBlt"] = test_df["GarageYrBlt"].fillna("None")

### 车库内饰 GarageFinish 训练集缺失81，测试集缺失78-》众数插补
GarageType_mode = train_df["GarageFinish"].mode()[0]
train_df.loc[:,"GarageFinish"] = train_df["GarageFinish"].fillna(GarageType_mode)
test_df.loc[:,"GarageFinish"] = test_df["GarageFinish"].fillna(GarageType_mode)

### 车壳大小以停车数量表示 GarageCars 测试集缺失1-》 没有可能是没车库，用0插补
test_df.loc[:,"GarageCars"] = test_df["GarageFinish"].fillna(0)

### 车库面积 GarageArea 测试集缺失1-》 没有可能是没车库，用0插补
test_df.loc[:,"GarageCars"] = test_df["GarageFinish"].fillna(0)

### 车库质量 GarageQual 训练集缺失81，测试集缺失78
GarageQual_mode = train_df["GarageQual"].mode()[0]
train_df.loc[:,"GarageQual"] = train_df["GarageQual"].fillna(GarageQual_mode)
test_df.loc[:,"GarageQual"] = test_df["GarageQual"].fillna(GarageQual_mode)

### 车库条件 GarageCond 训练集缺失81，测试集缺失78
GarageCond_mode = train_df["GarageCond"].mode()[0]
train_df.loc[:,"GarageCond"] = train_df["GarageCond"].fillna(GarageCond_mode)
test_df.loc[:,"GarageCond"] = test_df["GarageCond"].fillna(GarageCond_mode)

/var/folders/d3/j2fy35092td61w0znws_020w0000gn/T/ipykernel_66816/3835375126.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2003.0 1976.0 2001.0 ... 1941.0 1950.0 1965.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  train_df.loc[:,"GarageYrBlt"] = train_df["GarageYrBlt"].fillna("None")
/var/folders/d3/j2fy35092td61w0znws_020w0000gn/T/ipykernel_66816/3835375126.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1961.0 1958.0 1997.0 ... 1960.0 'None' 1993.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_df.loc[:,"GarageYrBlt"] = test_df["GarageYrBlt"].fillna("None")
/var/folders/d3/j2fy35092td61w0znws_020w0000gn/T/ipykernel_66816/3835375126.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future err

### 游泳池与其他

In [9]:
# 游泳池质量 PoolQC 训练集缺失1453，测试集缺失1456, 缺失量过多, 用unknown表示
train_df.loc[:,"PoolQC"] = train_df["PoolQC"].fillna("None")
test_df.loc[:,"PoolQC"] = test_df["PoolQC"].fillna("None")

# 围栏质量 Fence 训练集缺失1179，测试集缺失1169, 缺失量过多, 用unknown表示
train_df.loc[:,"Fence"] = train_df["Fence"].fillna("None")
test_df.loc[:,"Fence"] = test_df["Fence"].fillna("None")

# 其它条件中未包含部分的特性 MiscFeature 训练集缺失1406，测试集缺失1408, 缺失量过多, 用unknown表示
train_df.loc[:,"MiscFeature"] = train_df["MiscFeature"].fillna("None")
test_df.loc[:,"MiscFeature"] = test_df["MiscFeature"].fillna("None")

# SaleType 测试集缺失1
SaleType_mode = train_df["SaleType"].mode()[0]
test_df.loc[:,"SaleType"] = test_df["SaleType"].fillna(SaleType_mode)

In [10]:
# ## 查看插补后有无缺失值：
# pd.set_option('display.max_rows', None)
# print(train_df.isnull().sum())
# print(test_df.isnull().sum())

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr

## 异常值处理

## 特征构造